In [11]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.remove('c:\\Users\\Pandas\\taxi_demand_predictor\\notebooks')
sys.path.append('c:\\Users\\Pandas\\taxi_demand_predictor\\')

ValueError: list.remove(x): x not in list

In [ ]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73135,3.0,4.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,3.0,...,29.0,15.0,4.0,12.0,7.0,11.0,4.0,2022-10-27,265,12.0
73136,9.0,4.0,3.0,3.0,2.0,4.0,2.0,6.0,1.0,3.0,...,29.0,17.0,12.0,4.0,5.0,8.0,9.0,2022-10-28,265,3.0
73137,7.0,6.0,3.0,4.0,3.0,5.0,7.0,6.0,5.0,10.0,...,29.0,13.0,9.0,10.0,5.0,10.0,7.0,2022-10-29,265,6.0
73138,6.0,5.0,8.0,6.0,6.0,0.0,1.0,2.0,8.0,6.0,...,8.0,10.0,7.0,3.0,3.0,6.0,2.0,2022-10-30,265,10.0


In [12]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(40545, 674)
y_test.shape=(40545,)


In [13]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [14]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2023-07-17 15:29:26,120] A new study created in memory with name: no-name-bb993707-bcc3-4feb-bd1e-2c572b8aacc8


[LightGBM] [Warning] feature_fraction is set=0.9417045297913684, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9417045297913684
[LightGBM] [Warning] bagging_fraction is set=0.27870389590306016, subsample=1.0 will be ignored. Current value: bagging_fraction=0.27870389590306016
[LightGBM] [Warning] feature_fraction is set=0.9417045297913684, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9417045297913684
[LightGBM] [Warning] bagging_fraction is set=0.27870389590306016, subsample=1.0 will be ignored. Current value: bagging_fraction=0.27870389590306016


[I 2023-07-17 15:29:35,881] Trial 0 finished with value: 3.4922321166040358 and parameters: {'num_leaves': 208, 'feature_fraction': 0.9417045297913684, 'bagging_fraction': 0.27870389590306016, 'min_child_samples': 9}. Best is trial 0 with value: 3.4922321166040358.


[LightGBM] [Warning] feature_fraction is set=0.95806104479271, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.95806104479271
[LightGBM] [Warning] bagging_fraction is set=0.9606205582054714, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9606205582054714
[LightGBM] [Warning] feature_fraction is set=0.95806104479271, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.95806104479271
[LightGBM] [Warning] bagging_fraction is set=0.9606205582054714, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9606205582054714


[I 2023-07-17 15:29:41,523] Trial 1 finished with value: 3.3370644957510938 and parameters: {'num_leaves': 112, 'feature_fraction': 0.95806104479271, 'bagging_fraction': 0.9606205582054714, 'min_child_samples': 35}. Best is trial 1 with value: 3.3370644957510938.


...

[I 2023-07-17 15:29:44,797] Trial 2 finished with value: 3.70600502132075 and parameters: {'num_leaves': 46, 'feature_fraction': 0.905694329460017, 'bagging_fraction': 0.44586093984145336, 'min_child_samples': 82}. Best is trial 1 with value: 3.3370644957510938.


[LightGBM] [Warning] feature_fraction is set=0.7928785105346245, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7928785105346245
[LightGBM] [Warning] bagging_fraction is set=0.7413057187654022, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7413057187654022
[LightGBM] [Warning] feature_fraction is set=0.7928785105346245, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7928785105346245
[LightGBM] [Warning] bagging_fraction is set=0.7413057187654022, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7413057187654022


[I 2023-07-17 15:29:51,343] Trial 3 finished with value: 3.4050934867057707 and parameters: {'num_leaves': 155, 'feature_fraction': 0.7928785105346245, 'bagging_fraction': 0.7413057187654022, 'min_child_samples': 43}. Best is trial 1 with value: 3.3370644957510938.


[LightGBM] [Warning] feature_fraction is set=0.8165698851035628, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8165698851035628
[LightGBM] [Warning] bagging_fraction is set=0.7565556163327942, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7565556163327942
[LightGBM] [Warning] feature_fraction is set=0.8165698851035628, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8165698851035628
[LightGBM] [Warning] bagging_fraction is set=0.7565556163327942, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7565556163327942


[I 2023-07-17 15:30:00,902] Trial 4 finished with value: 3.356321646146511 and parameters: {'num_leaves': 219, 'feature_fraction': 0.8165698851035628, 'bagging_fraction': 0.7565556163327942, 'min_child_samples': 10}. Best is trial 1 with value: 3.3370644957510938.


In [15]:

best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 112, 'feature_fraction': 0.95806104479271, 'bagging_fraction': 0.9606205582054714, 'min_child_samples': 35}


In [16]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.95806104479271, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.95806104479271
[LightGBM] [Warning] bagging_fraction is set=0.9606205582054714, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9606205582054714


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000002BDC1418AE0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.9606205582054714,
                               feature_fraction=0.95806104479271,
                               min_child_samples=35, num_leaves=112))])

In [17]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.4254


In [27]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=12085,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)


In [29]:
plot_one_sample(
    example_id=7194,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [24]:
X_test.average_rides_last_4_weeks.describe()

count    40545.000000
mean        11.927790
std         35.337048
min          0.000000
25%          0.000000
50%          0.000000
75%          2.500000
max        463.000000
Name: average_rides_last_4_weeks, dtype: float64

In [28]:
X_test.query('average_rides_last_4_weeks > 200')

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,average_rides_last_4_weeks
7194,247.0,186.0,145.0,83.0,64.0,33.0,43.0,68.0,93.0,170.0,...,196.0,189.0,219.0,148.0,239.0,298.0,347.0,2022-06-04,48,274.00
7195,277.0,216.0,167.0,113.0,107.0,43.0,49.0,58.0,64.0,126.0,...,225.0,261.0,230.0,169.0,248.0,296.0,299.0,2022-06-05,48,278.50
7201,318.0,250.0,167.0,155.0,100.0,44.0,66.0,71.0,77.0,123.0,...,213.0,215.0,249.0,194.0,264.0,268.0,297.0,2022-06-11,48,288.25
7202,282.0,233.0,179.0,131.0,102.0,44.0,57.0,70.0,68.0,138.0,...,209.0,204.0,215.0,149.0,226.0,284.0,263.0,2022-06-12,48,287.50
7208,265.0,189.0,147.0,123.0,89.0,44.0,48.0,70.0,97.0,126.0,...,188.0,215.0,236.0,203.0,258.0,299.0,274.0,2022-06-18,48,288.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38087,241.0,333.0,266.0,180.0,57.0,11.0,8.0,26.0,52.0,57.0,...,109.0,142.0,202.0,245.0,254.0,268.0,282.0,2022-10-22,249,243.50
38088,219.0,309.0,253.0,197.0,96.0,10.0,11.0,25.0,20.0,32.0,...,172.0,192.0,237.0,222.0,275.0,272.0,239.0,2022-10-23,249,223.00
38093,259.0,180.0,88.0,53.0,15.0,11.0,19.0,52.0,74.0,77.0,...,108.0,172.0,164.0,200.0,233.0,207.0,193.0,2022-10-28,249,222.50
38094,250.0,334.0,278.0,185.0,45.0,11.0,9.0,28.0,34.0,51.0,...,119.0,160.0,207.0,192.0,212.0,258.0,175.0,2022-10-29,249,255.00
